In [1]:
from datasets import load_dataset

dataset = load_dataset("imdb")["train"].train_test_split(test_size=0.05, train_size=0.1)


Found cached dataset imdb (/Users/chris-rotondo/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
dataset["test"].to_pandas()

,text,label
0,Well after three times through I still have no...,0
1,This movie is AWESOME. I watched it the other ...,0
2,I saw this movie when it came out when I was 1...,1
3,I can't come up with appropriate enough words ...,0
4,An unusually straight-faced actioner played by...,1
...,...,...
1245,I'm not going to waste my time writing an essa...,0
1246,There is something about Pet Sematary that I n...,1
1247,I have always had the philosophy that every si...,1
1248,"For all the Homicide junkies out there, this m...",1


In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(data):
    return tokenizer(data["text"], padding=True, truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

In [4]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [5]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    evaluation_strategy="steps", 
    eval_steps=101,             
    logging_dir="./logs",
    logging_steps=5,          
    save_steps=100,
    output_dir="./results",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train() 

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


In [ ]:
model.save_pretrained("./sentiment_model")
tokenizer.save_pretrained("./sentiment_model")